# Emerging Technologies – Problems 1–5

This notebook will contain solutions to Problems 1–5 for the Emerging Technologies assessment.

## Problem 1: Generating Random Boolean Functions

Given four Boolean inputs and one Boolean output, the task is to generate a function that is either constant (always the same output) or balanced (outputs True for exactly half of all possible inputs). The function must be randomly chosen from all such valid functions and returned as a callable.

**Background**

A Boolean function maps a set of Boolean inputs to a single Boolean output. A function is *constant* if it always returns the same value (always True or always False), regardless of input. A function is *balanced* if it returns True for exactly half of all possible input combinations and False for the other half. Distinguishing between constant and balanced functions is central to the Deutsch–Jozsa algorithm, as quantum algorithms can determine this global property with fewer queries than classical approaches.

**Design approach**

There are 2⁴ = 16 possible input combinations for four Boolean variables. To generate a constant function, we select a single output value (True or False) and return it for all inputs. For a balanced function, we randomly select exactly 8 of the 16 inputs to map to True, with the remainder mapping to False; this mapping can be stored as a set or lookup table. The function will be returned as a callable f(a, b, c, d) constructed from the chosen mapping.

**Implementation notes**

The function randomly decides whether to generate a constant or balanced Boolean function. For the constant case, it picks a single output bit and returns it for all inputs. For the balanced case, it enumerates all 16 possible input tuples, randomly selects 8 to map to True, stores them in a set, and returns True exactly when the input tuple is in that set. Using a set allows for fast membership testing.

**Correctness**

The constant branch yields either 0 or 16 True outputs across all inputs, depending on the chosen output bit. The balanced branch yields exactly 8 True outputs by construction, since 8 input tuples are mapped to True. This ensures the function always satisfies the Deutsch–Jozsa promise of being either constant or balanced.

**Sanity check**

The sanity check cell generates a random function and evaluates it on all 16 possible inputs, counting the number of True outputs. The result should be 0, 8, or 16, confirming the function is constant or balanced. The assert ensures this property holds for every generated function.

In [1]:
import random
import itertools

def random_constant_balanced():
    """
    Returns a randomly chosen Boolean function of four inputs that is guaranteed
    to be either constant or balanced (as required by Deutsch–Jozsa).
    """
    if random.choice([True, False]):
        # Constant function
        output_value = random.choice([False, True])
        def f(a, b, c, d):
            """Constant function: always returns {}.""".format(output_value)
            return bool(output_value)
        return f
    else:
        # Balanced function
        inputs = list(itertools.product([False, True], repeat=4))
        true_inputs = set(random.sample(inputs, 8))
        def f(a, b, c, d):
            """Balanced function: returns True for exactly 8 of 16 inputs."""
            a, b, c, d = bool(a), bool(b), bool(c), bool(d)
            return (a, b, c, d) in true_inputs
        return f

In [2]:
# Sanity check (optional)
f = random_constant_balanced()
inputs = list(itertools.product([False, True], repeat=4))
true_count = sum(f(*inp) for inp in inputs)
print(f"True outputs: {true_count}")
assert true_count in {0, 8, 16}

True outputs: 0


## Problem 2: Classical Testing for Function Type

Given a Boolean function f(a, b, c, d) that is promised to be either *constant* (same output for all 16 inputs) or *balanced* (True for exactly 8 inputs and False for the other 8), the task is to determine which type it is using classical evaluation only. The function must return the string "constant" or "balanced" accordingly.

**Classical cost / efficiency**

For four Boolean inputs there are $2^4 = 16$ possible input combinations. A classical algorithm must query the oracle (evaluate $f$) on distinct inputs and inspect the outputs.

- **Early exit on mixed outputs.** If at any point we observe both `True` and `False` among the outputs collected so far, the function cannot be constant—it must be balanced (a constant function produces only one output value). We can return `"balanced"` immediately.
- **Worst-case for proving constant (pigeonhole principle).** Suppose every query so far has returned the same value. After seeing 8 identical outputs we still cannot be certain: a balanced function also has exactly 8 inputs that map to each value, so it is possible all 8 belong to the same half. However, once we see a 9th identical output, a balanced function is ruled out—it has only 8 inputs for any given output value. Therefore, 9 queries producing the same result suffice to guarantee the function is constant.
- **Worst-case number of queries: 9.** In the absolute worst case we must evaluate $f$ on 9 distinct inputs before we can be 100 % certain. (Best case is 2 queries—one returning `True` and the other `False`.)
- **Time complexity.** Because the domain is fixed at 16 inputs, the number of oracle calls is $O(1)$ with respect to problem size. In the general $n$-bit setting (i.e. $2^n$ possible inputs), the worst-case classical cost is $O(2^n)$ queries—specifically $2^{n-1}+1$. The Deutsch–Jozsa quantum algorithm solves the same problem with a single query.

**Planned approach**

The implementation will iterate over distinct 4-bit inputs, evaluating $f$ on each:

1. **Maintain a record of observed outputs** (both a count and the first output seen).
2. **If both** `True` **and** `False` **have been observed** → return `"balanced"` immediately (early exit).
3. **If 9 inputs have all produced the same output** → return `"constant"` (by the pigeonhole argument above, a balanced function cannot have 9 inputs mapping to one value).

This strategy is worst-case optimal under the constant-or-balanced promise: no classical deterministic algorithm can guarantee a correct answer in fewer than 9 queries.

In [2]:
def determine_constant_balanced(f):
    """
    Determine whether a 4-input Boolean function f is constant or balanced.

    Parameters
    ----------
    f : callable
        A function taking four Boolean inputs and returning a Boolean output.
        It is promised to be either constant or balanced.

    Returns
    -------
    str
        "constant" or "balanced"
    """
    inputs = list(itertools.product([False, True], repeat=4))
    first_out = bool(f(*inputs[0]))
    same_count = 1

    for inp in inputs[1:]:
        out = bool(f(*inp))
        if out != first_out:
            return "balanced"
        same_count += 1
        if same_count == 9:
            return "constant"

    raise ValueError(
        "Could not determine function type — the constant/balanced promise may be violated."
    )

In [3]:
# Minimal verification (Problem 2)
inputs = list(itertools.product([False, True], repeat=4))

for _ in range(5):
    f = random_constant_balanced()
    result = determine_constant_balanced(f)
    true_count = sum(bool(f(*inp)) for inp in inputs)
    print(f"true_count={true_count} -> {result}")
    if result == "balanced":
        assert true_count == 8
    elif result == "constant":
        assert true_count in (0, 16)

true_count=8 -> balanced
true_count=8 -> balanced
true_count=16 -> constant
true_count=16 -> constant
true_count=8 -> balanced
